In [ ]:
import dask.dataframe as dd

df = dd.read_parquet("hf://datasets/community-datasets/generics_kb/generics_kb/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
df.head()

,source,term,quantifier_frequency,quantifier_number,generic_sentence,score
0,Waterloo,a.active replication,,,A.Active replication requires all members to e...,0.024261
1,Waterloo,a.net income,,,A.Net income is divided by net sales.,0.139061
2,Waterloo,a.p.e motorcycle,,,A.P.E motorcycle babes specializes in bondage ...,0.117522
3,ARC,a.sound wave,,,A.sound waves can reflect.,0.120108
4,ARC,a.their image,,,A.their images are inverted.,0.028758


In [ ]:
df_pd = df.compute()
df_pd.head()

,source,term,quantifier_frequency,quantifier_number,generic_sentence,score
0,Waterloo,a.active replication,,,A.Active replication requires all members to e...,0.024261
1,Waterloo,a.net income,,,A.Net income is divided by net sales.,0.139061
2,Waterloo,a.p.e motorcycle,,,A.P.E motorcycle babes specializes in bondage ...,0.117522
3,ARC,a.sound wave,,,A.sound waves can reflect.,0.120108
4,ARC,a.their image,,,A.their images are inverted.,0.028758


In [ ]:
# prompt: check if any of the generic_sentence begin with ['all', 'every', 'most', 'many', 'some', 'few', 'no']

# Define the list of allowed quantifiers
allowed_quantifiers = ['all', 'every', 'most', 'many', 'some', 'few', 'no']

# Check if the generic_sentence starts with any of the allowed quantifiers
def starts_with_quantifier(sentence, quantifiers):
  if isinstance(sentence, str):
    for quantifier in quantifiers:
      if sentence.lower().strip().startswith(quantifier):
        return True
  return False

# Apply the function to the 'generic_sentence' column
df_pd['starts_with_quantifier'] = df_pd['generic_sentence'].apply(
    lambda x: starts_with_quantifier(x, allowed_quantifiers)
)

# Display sentences that start with a quantifier
print("\nSentences starting with a quantifier:")
print(df_pd[df_pd['starts_with_quantifier']][['generic_sentence', 'score', 'starts_with_quantifier']])

# Count how many sentences start with a quantifier
num_starting_with_quantifier = df_pd['starts_with_quantifier'].sum()
print(f"\nNumber of generic sentences starting with one of the allowed quantifiers: {num_starting_with_quantifier}")

# Check if any generic_sentence begins with the specified quantifiers
any_starts_with_quantifier = df_pd['starts_with_quantifier'].any()

print(f"\nDo any of the filtered generic sentences begin with ['all', 'every', 'most', 'many', 'some', 'few', 'no']? {any_starts_with_quantifier}")



Sentences starting with a quantifier:
                                          generic_sentence     score  \
50       Every AACA member is invited to join the judgi...  0.024421   
54       Some AANR clubs have membership policies that ...  0.191635   
215                               Most aardvarks eat ants.  0.732159   
216                   Most aardvarks eat first solid food.  0.374309   
217                               Most aardvarks eat food.  0.447961   
...                                                    ...       ...   
3432839                  Some zooplankton is near bottoms.  0.452655   
3432840                 Some zooplankton reaches maturity.  0.596575   
3432875          Many zoospores are produced in each cell.  0.753576   
3432953                   Some zygospores undergo meiosis.  0.757018   
3432995  All Zymark employees are responsible for respe...  0.049532   

         starts_with_quantifier  
50                         True  
54                         T

In [ ]:
# prompt: create a pd dataframe with columns called `text` (as the generic_sentence above) and `original_quantifier` (as the quanitifier the semtence starts with)

import pandas as pd

# Create a new DataFrame with the desired columns
df_quantified = df_pd[df_pd['starts_with_quantifier']].copy()
df_quantified = df_quantified.rename(columns={'generic_sentence': 'text'})

# Function to extract the starting quantifier
def extract_quantifier(sentence, quantifiers):
  if isinstance(sentence, str):
    for quantifier in quantifiers:
      if sentence.lower().strip().startswith(quantifier):
        # Extract the matched quantifier
        return quantifier.lower().strip()
  return None

# Apply the function to the 'text' column to create 'original_quantifier'
df_quantified['original_quantifier'] = df_quantified['text'].apply(
    lambda x: extract_quantifier(x, allowed_quantifiers)
)

# Display the new DataFrame
print("\nDataFrame with 'text' and 'original_quantifier':")
print(df_quantified.head())


DataFrame with 'text' and 'original_quantifier':
       source         term quantifier_frequency quantifier_number  \
50   Waterloo  aaca member                                  every   
54   Waterloo    aanr club                                   some   
215       ARC     aardvark                                   Most   
216       ARC     aardvark                                   Most   
217       ARC     aardvark                                   Most   

                                                  text     score  \
50   Every AACA member is invited to join the judgi...  0.024421   
54   Some AANR clubs have membership policies that ...  0.191635   
215                           Most aardvarks eat ants.  0.732159   
216               Most aardvarks eat first solid food.  0.374309   
217                           Most aardvarks eat food.  0.447961   

     starts_with_quantifier original_quantifier  
50                     True               every  
54                     Tru

In [ ]:
df_filtered = df_quantified[['text', 'original_quantifier']]
df_filtered.head()

,text,original_quantifier
50,Every AACA member is invited to join the judgi...,every
54,Some AANR clubs have membership policies that ...,some
215,Most aardvarks eat ants.,most
216,Most aardvarks eat first solid food.,most
217,Most aardvarks eat food.,most


In [ ]:
len(df_filtered)

653866

In [ ]:
def remove_the(text):
  if isinstance(text, str):
    return text.replace("the ", "").replace(" The ", "")
  return text

df_filtered['text_without_the'] = df_filtered['text'].apply(remove_the)

print(df_filtered[['text', 'text_without_the']].to_string())

/tmp/ipython-input-8-3917814513.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['text_without_the'] = df_filtered['text'].apply(remove_the)


Buffered data was truncated after reaching the output size limit.

In [ ]:
stripped_txt_lst, quantifier_lst = [], []

i = 0
for index, row in df_filtered.iterrows():
  i += 1
  new_row = row['text_without_the'].split()[1:]
  if i < 20:
    print(f"Original Text: {row['text']}")
    print(f"Original Quantifier: {row['original_quantifier']}")
    print(f"New Text: {' '.join(new_row)}")
    print("-" * 20)
  stripped_txt_lst.append(' '.join(new_row))
  quantifier_lst.append(row['original_quantifier'])

Original Text: Every AACA member is invited to join the judging ranks.
Original Quantifier: every
New Text: AACA member is invited to join judging ranks.
--------------------
Original Text: Some AANR clubs have membership policies that focus on couples and families.
Original Quantifier: some
New Text: AANR clubs have membership policies that focus on couples and families.
--------------------
Original Text: Most aardvarks eat ants.
Original Quantifier: most
New Text: aardvarks eat ants.
--------------------
Original Text: Most aardvarks eat first solid food.
Original Quantifier: most
New Text: aardvarks eat first solid food.
--------------------
Original Text: Most aardvarks eat food.
Original Quantifier: most
New Text: aardvarks eat food.
--------------------
Original Text: Most aardvarks eat insects.
Original Quantifier: most
New Text: aardvarks eat insects.
--------------------
Original Text: Most aardvarks eat solid food.
Original Quantifier: most
New Text: aardvarks eat solid food

In [ ]:
import pandas as pd

data = {'Stripped Text': stripped_txt_lst, 'Original quantifier': quantifier_lst}
result_df = pd.DataFrame(data)
print(result_df.head())

                                       Stripped Text Original quantifier
0      AACA member is invited to join judging ranks.               every
1  AANR clubs have membership policies that focus...                some
2                                aardvarks eat ants.                most
3                    aardvarks eat first solid food.                most
4                                aardvarks eat food.                most


In [ ]:
seed = 42

result_df_shuffled = result_df.sample(frac=1, random_state=seed).reset_index(drop=True)
print(result_df_shuffled.head())

                                       Stripped Text Original quantifier
0  lawyers seem to fear that taking first step to...                some
1  people also gather, cook, and eat pads of pric...                some
2                pollen attracts beneficial insects.                most
3                               students use forces.                some
4      behavioral patterns are exhibited by possums.                some


In [ ]:
holdout_df = result_df_shuffled.sample(frac=0.05, random_state=seed)
remaining_df = result_df_shuffled.drop(holdout_df.index)

print("Holdout DataFrame:")
print(holdout_df.head())
print("\nRemaining DataFrame:")
print(remaining_df.head())

print(f"\nTotal rows in original result_df_shuffled: {len(result_df_shuffled)}")
print(f"Rows in holdout_df: {len(holdout_df)}")
print(f"Rows in remaining_df: {len(remaining_df)}")
print(f"Sum of holdout_df and remaining_df rows: {len(holdout_df) + len(remaining_df)}")

Holdout DataFrame:
                                            Stripped Text Original quantifier
282960  motorcyclists install a modulator on their hea...                some
424248        buffaloes are in private small stock herds.                most
461906  people use more complex metrics which try to s...                many
582995                      gut bacteria cause infection.                most
45975                         native ladybugs eat aphids.                most

Remaining DataFrame:
                                       Stripped Text Original quantifier
0  lawyers seem to fear that taking first step to...                some
1  people also gather, cook, and eat pads of pric...                some
2                pollen attracts beneficial insects.                most
3                               students use forces.                some
4      behavioral patterns are exhibited by possums.                some

Total rows in original result_df_shuffled: 653866
Ro

In [ ]:
holdout_df.to_csv('GenericsKB_dev.csv', index=False)
remaining_df.to_csv('GenericsKB_train.csv', index=False)

In [ ]:
from google.colab import files

files.download('GenericsKB_dev.csv')
files.download('GenericsKB_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>